# Synthetic Data Clustering

## Vectorisation Clustering

In [1]:
# imports
import numpy as np
import pandas as pd
from statistics import mean
from data_utils.generate_synthetic_data import make_point_clouds
from gtda.homology import VietorisRipsPersistence
from data_utils.vectorisation_methods import get_persistence_landscapes, get_betti_curves, get_persistence_images
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

import warnings
warnings.filterwarnings("ignore")

In this notebook we cluster the vector representations of the persistence diagrams of synthetic data for varying levels of noise.

In [2]:
# Initiate parameters for clustering with varying noise
noise = [0, 1, 2, 3, 4, 5, 10]
n_samples_per_class = 10
homology_dimensions = [0, 1, 2]
n_clusters = 3

landscape_rand = [None] * len(noise)
betti_rand = [None] * len(noise)
image_rand = [None] * len(noise)

km = KMeans(n_clusters=3, init='k-means++')

In [3]:
# calculate adjusted rand scores for clustering of data with varying noise
for i, n in enumerate(noise):
    # Create synthetic data of 10 samples of 4 classes, circles, spheres, tori and random point clouds
    point_clouds, labels = make_point_clouds(n_samples_per_class, n_points=10, noise=n)
    # Compute persistence diagrams
    VR = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
    diagrams = VR.fit_transform(point_clouds)
    # Compute persistence landscapes
    p_landscapes = get_persistence_landscapes(point_clouds, diagrams, n_layers=2, n_bins=50)
    # Compute betti curves
    betti_curves = get_betti_curves(point_clouds, diagrams, n_bins=100)
    # Compute persistence images
    p_images = get_persistence_images(point_clouds, diagrams, n_bins=10)
    # predict labels
    landscape_preds = km.fit_predict(p_landscapes)
    betti_preds = km.fit_predict(betti_curves)
    image_preds = km.fit_predict(p_images)
    # Compute rand score for each clustering
    landscape_rand[i] = adjusted_rand_score(labels, landscape_preds)
    betti_rand[i] = adjusted_rand_score(labels, betti_preds)
    image_rand[i] = adjusted_rand_score(labels, image_preds)

# print ARI scores in table
vector_scores = pd.DataFrame({'noise': noise,
                              'PL score': landscape_rand,
                              'PI score': image_rand,
                              'BC_score': betti_rand}).set_index('noise')
print(vector_scores)

       PL score  PI score  BC_score
noise                              
0      1.000000  1.000000  1.000000
1      1.000000  0.898170  0.274136
2      1.000000  1.000000  0.137032
3      1.000000  1.000000  0.077368
4      0.898170  0.898170  0.109962
5      0.898170  0.698192  0.109039
10     0.500717  0.440262  0.126139


We compare the clustering output of persistence landscapes and persistence images by repeating the experiment on variations of the simulated data.

In [4]:
def persistence_comparison(homology_dimensions: list, noise: int, iters: int):
    comparison = []
    landscape_scores = []
    image_scores = []
    # calculate
    for _ in range(iters):
        # initialise Persistent Homology
        VR = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
        # generate data with set noise level
        point_clouds, labels = make_point_clouds(n_samples_per_class, n_points=10, noise=noise)
        # create persistence diagrams
        diagrams = VR.fit_transform(point_clouds)
        # create persistence landscape and image vectors
        p_landscapes = get_persistence_landscapes(point_clouds=point_clouds,
                                                  persistence_diagrams=diagrams,
                                                  n_layers=2,
                                                  n_bins=50)
        p_images = get_persistence_images(point_clouds=point_clouds,
                                          persistence_diagrams=diagrams,
                                          n_bins=10)
        # cluster based on vectors
        landscape_preds =  km.fit_predict(p_landscapes)
        image_preds = km.fit_predict(p_images)
        # calculate adjusted rand score for each vectorization
        landscape_score = adjusted_rand_score(labels, landscape_preds)
        image_score = adjusted_rand_score(labels, image_preds)
        # append scores to list
        landscape_scores.append(landscape_score)
        image_scores.append(image_score)
        # append 1 if PLs outperform PIs
        if image_score < landscape_score:
            comparison.append(1)
        else:
            comparison.append(0)
    print(f"For noise = {noise}, persistence landscapes outperform persistence images "
          f"{round(mean(comparison) * 100, 2)}% of the time.")
    print(f" Average Adjusted Rand Score for Persistence Landscapes: {round(mean(landscape_scores), 3)}")
    print(f" Std. Adjusted Rand Score for Persistence Landscapes: {round(np.std(landscape_scores), 3)}")
    print(f" Average Adjusted Rand Score for Persistence Images: {round(mean(image_scores), 3)}")
    print(f" Std. Adjusted Rand Score for Persistence Images: {round(np.std(image_scores), 3)}")

In [5]:
# example of persistence landscape/vector comparison for noise = 1.0

persistence_comparison(homology_dimensions=[0, 1, 2], noise=1.0, iters=100)

For noise = 1.0, persistence landscapes outperform persistence images 56.0% of the time.
 Average Adjusted Rand Score for Persistence Landscapes: 0.993
 Std. Adjusted Rand Score for Persistence Landscapes: 0.026
 Average Adjusted Rand Score for Persistence Images: 0.91
 Std. Adjusted Rand Score for Persistence Images: 0.094


## Persistence Diagram Clustering

In this section we demonstrate running the PD and PM K-means clustering algorithms on simulated data.

In [6]:
# imports
from pd_pm_kmeans import PD_KMeans, PM_KMeans
from data_utils.pd_pm_methods import *

In [7]:
# Create simulated data
point_clouds, labels = make_point_clouds(n_samples_per_class, n_points=10, noise=1.0)

# Create PDs from simulated data
diagrams = []

for pc in point_clouds:
    norm_pc = normalise_pc(pc)
    diag = get_pd(norm_pc)
    diagrams.append(diag)

In [8]:
# Clustering in Persistence Diagram Space
km = PD_KMeans(n_clusters=3, init='kmeans++', random_state=123)
pd_preds = km.fit(diagrams)
print(f'PD ARI score: {adjusted_rand_score(labels, pd_preds)}')

PD ARI score: 1.0


Clustering Persistence Measures

In [9]:
# get appropriate grid_width from list of PDs
grid_width = get_grid_width(diagrams)

# create list of PMs from PDs
mesrs = []
for diag in diagrams:
    concat_diag = np.concatenate(diag)
    mesr, _ = diag_to_mesr(concat_diag, unit_mass=1, grid_width=grid_width)
    mesrs.append(mesr)

pm_km = PM_KMeans(n_clusters=3, init='kmeans++', grid_width=grid_width)
pm_preds = pm_km.fit(mesrs)

print(f'PM ARI Score: {adjusted_rand_score(labels, pm_preds)}')

PM ARI Score: 1.0
